In [20]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

import numpy as np

tf.get_logger().setLevel('ERROR')

In [21]:
!pip install tensorflow-privacy

In [22]:
import tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [23]:
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

assert train_data.min() == 0.
assert train_data.max() == 1.
assert test_data.min() == 0.
assert test_data.max() == 1.

In [42]:
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 1
learning_rate = 0.25

batch_size=1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [43]:
train_data.shape

(60000, 28, 28, 1)

In [44]:
train_labels.shape

(60000, 10)

In [45]:
tr=train_data[:2,:,:,:]
te=train_data[:2,:,:,:]
tr_l=train_labels[:2,:]
te_l=test_labels[:2,:]

In [46]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [47]:
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [48]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
epochs=1
model.fit(tr, tr_l,
          epochs=epochs,
          validation_data=(te, te_l),
          batch_size=1)

Train on 2 samples, validate on 2 samples
1/2 [==============>...............] - ETA: 0s - loss: 2.5683 - acc: 0.0000e+00

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2/2 [==============================] - 6s 5s/sample - loss: 85.1717 - acc: 0.0000e+00 - val_loss: 1711.7944 - val_acc: 0.0000e+00


In [49]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=tr.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

DP-SGD with sampling rate = 50% and noise_multiplier = 1.3 iterated over 2 steps satisfies differential privacy with eps = 3.74 and delta = 1e-05.
The optimal RDP order is 5.0.


(3.741572930611243, 5.0)

In [ ]:
import tensorflow as tf
from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras

# Select your differentially private optimizer
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

# Select your loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

# Compile your model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Fit your model
model.fit(train_data, train_labels,
  epochs=epochs,
  validation_data=(test_data, test_labels),
  batch_size=batch_size)
  